Integrantes  
Bryann Alfaro  
Raul Jimenez  
Donaldo Garcia

In [1]:
#Import 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer


# Parte 1 – Ingeniería de características

### Exploracion de caracteristicas

In [2]:
#Read the csv into a df
df = pd.read_csv('data/completeSpamAssassin.csv')
df2 = pd.read_csv('data/enronSpamSubset.csv')
#Print the first 5 rows of the df
df.head()

,Unnamed: 0,Body,Label
0,0,\r\nSave up to 70% on Life Insurance.\r\nWhy S...,1
1,1,1) Fight The Risk of Cancer!\r\nhttp://www.adc...,1
2,2,1) Fight The Risk of Cancer!\r\nhttp://www.adc...,1
3,3,##############################################...,1
4,4,I thought you might like these:\r\n1) Slim Dow...,1


In [3]:
df2.head()

,Unnamed: 0.1,Unnamed: 0,Body,Label
0,2469,2469,Subject: stock promo mover : cwtd\r\n * * * ur...,1
1,5063,5063,Subject: are you listed in major search engine...,1
2,12564,12564,"Subject: important information thu , 30 jun 20...",1
3,2796,2796,Subject: = ? utf - 8 ? q ? bask your life with...,1
4,1468,1468,"Subject: "" bidstogo "" is places to go , things...",1


In [4]:
#Eliminar las columnas que no se van a utilizar Unnamed en ambos
df = df.drop(['Unnamed: 0'], axis=1)
df2 = df2.drop(['Unnamed: 0'], axis=1)
df2 = df2.drop(['Unnamed: 0.1'], axis=1)

In [5]:
#Join the two dataframes
df = df.append(df2, ignore_index=True)

C:\Users\Bryann\AppData\Local\Temp\ipykernel_30152\3655632441.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df2, ignore_index=True)


In [6]:
df.head()

,Body,Label
0,\r\nSave up to 70% on Life Insurance.\r\nWhy S...,1
1,1) Fight The Risk of Cancer!\r\nhttp://www.adc...,1
2,1) Fight The Risk of Cancer!\r\nhttp://www.adc...,1
3,##############################################...,1
4,I thought you might like these:\r\n1) Slim Dow...,1


In [7]:
df.shape

(16046, 2)

In [8]:
#lowercase of column names
df.columns = df.columns.str.lower()
df.head()

,body,label
0,\r\nSave up to 70% on Life Insurance.\r\nWhy S...,1
1,1) Fight The Risk of Cancer!\r\nhttp://www.adc...,1
2,1) Fight The Risk of Cancer!\r\nhttp://www.adc...,1
3,##############################################...,1
4,I thought you might like these:\r\n1) Slim Dow...,1


In [9]:
#Eliminar na
df = df.dropna()
df.shape

(16045, 2)

### Preprocesamiento 

In [10]:
# Remove stop words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

# Exclude stopwords with Python's list comprehension and pandas.DataFrame.apply.
df['body'] = df['body'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
print(df)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Bryann\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


                                                    body  label
0      Save 70% Life Insurance. Why Spend More Than Y...      1
1      1) Fight The Risk Cancer! http://www.adclick.w...      1
2      1) Fight The Risk Cancer! http://www.adclick.w...      1
3      ##############################################...      1
4      I thought might like these: 1) Slim Down - Gua...      1
...                                                  ...    ...
16041  Subject: monday 22 nd oct louise , half hour f...      0
16042  Subject: missing bloomberg deals stephanie - b...      0
16043  Subject: eops salary survey questionnaire need...      0
16044  Subject: q 3 comparison hi louise , comparison...      0
16045  Subject: confidential folder safely pass infor...      0

[16045 rows x 2 columns]


In [11]:
# Remove punctuation
import string
df['body'] = df['body'].str.replace(f'[{string.punctuation}]', '')
print(df)

C:\Users\Bryann\AppData\Local\Temp\ipykernel_30152\1140702011.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['body'] = df['body'].str.replace(f'[{string.punctuation}]', '')


                                                    body  label
0      Save 70 Life Insurance Why Spend More Than You...      1
1      1 Fight The Risk Cancer httpwwwadclickwspcfmo3...      1
2      1 Fight The Risk Cancer httpwwwadclickwspcfmo3...      1
3          Adult Club   Offers FREE Membership     IN...      1
4      I thought might like these 1 Slim Down  Guaran...      1
...                                                  ...    ...
16041  Subject monday 22 nd oct louise  half hour fre...      0
16042  Subject missing bloomberg deals stephanie  bel...      0
16043  Subject eops salary survey questionnaire need ...      0
16044  Subject q 3 comparison hi louise  comparison f...      0
16045  Subject confidential folder safely pass inform...      0

[16045 rows x 2 columns]


In [12]:
# Turn every word to lowercase
df['body'] = df['body'].str.lower()
print(df)

                                                    body  label
0      save 70 life insurance why spend more than you...      1
1      1 fight the risk cancer httpwwwadclickwspcfmo3...      1
2      1 fight the risk cancer httpwwwadclickwspcfmo3...      1
3          adult club   offers free membership     in...      1
4      i thought might like these 1 slim down  guaran...      1
...                                                  ...    ...
16041  subject monday 22 nd oct louise  half hour fre...      0
16042  subject missing bloomberg deals stephanie  bel...      0
16043  subject eops salary survey questionnaire need ...      0
16044  subject q 3 comparison hi louise  comparison f...      0
16045  subject confidential folder safely pass inform...      0

[16045 rows x 2 columns]


In [13]:
# Expand contractions
import contractions
df['body'] = df['body'].apply(lambda x: ' '.join([contractions.fix(word) for word in x.split()]))
print(df)

                                                    body  label
0      save 70 life insurance why spend more than you...      1
1      1 fight the risk cancer httpwwwadclickwspcfmo3...      1
2      1 fight the risk cancer httpwwwadclickwspcfmo3...      1
3      adult club offers free membership instant acce...      1
4      i thought might like these 1 slim down guarant...      1
...                                                  ...    ...
16041  subject monday 22 nd oct louise half hour free...      0
16042  subject missing bloomberg deals stephanie beli...      0
16043  subject eops salary survey questionnaire need ...      0
16044  subject q 3 comparison hi louise comparison fi...      0
16045  subject confidential folder safely pass inform...      0

[16045 rows x 2 columns]


### Representacion de texto

In [15]:
#Make a Bag of N-Grams 

vectorizer = CountVectorizer(ngram_range=(1, 2), min_df=0.1 , max_df=0.8)
matrix_vect = vectorizer.fit_transform(df['body']).toarray()

#Create a dataframe with the matrix
df_bog = pd.DataFrame(matrix_vect, columns=vectorizer.get_feature_names_out())
df_bog.head()



,10,2000,2002,also,best,business,cc,click,com,company,...,to,today,us,use,want,way,well,work,would,you
0,0,0,0,0,3,0,0,2,0,0,...,0,0,1,0,0,0,0,0,0,2
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
3,0,0,0,0,3,0,0,1,0,0,...,1,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1


In [16]:
#Make a TF-IDF

vectorizer = TfidfVectorizer(min_df=0.1 , max_df=0.8, use_idf=True)
matrix_vect = vectorizer.fit_transform(df['body']).toarray()

#Create a dataframe with the matrix
df_tfidf = pd.DataFrame(matrix_vect, columns=vectorizer.get_feature_names_out())
df_tfidf.head()


,10,2000,2002,also,best,business,cc,click,com,company,...,to,today,us,use,want,way,well,work,would,you
0,0.0,0.0,0.0,0.0,0.458674,0.0,0.0,0.310028,0.0,0.0,...,0.000000,0.0,0.132523,0.000000,0.0,0.0,0.0,0.0,0.0,0.289067
1,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.293959,0.0,0.0,0.0,0.0,0.0,0.293278
2,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.368609,0.0,0.0,0.0,0.0,0.0,0.367755
3,0.0,0.0,0.0,0.0,0.272546,0.0,0.0,0.092110,0.0,0.0,...,0.097004,0.0,0.078746,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
4,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.299606,0.0,0.0,0.0,0.0,0.0,0.298912


# Parte 2 - Implementacion del modelo

### Separacion de datos

##### Modelo Bag of N words

In [17]:
#Split the data into train and test

target = df['label']

X_train, X_test, y_train, y_test = train_test_split(df_bog, target, test_size=0.3, random_state=42)

In [18]:
#Multinomial Naive Bayes model
model_naive_bog = MultinomialNB()
model_naive_bog.fit(X_train, y_train)

y_pred = model_naive_bog.predict(X_test)

#Evaluate the model
print("Accuracy: ", accuracy_score(y_test, y_pred))
print("Confusion Matrix: ", confusion_matrix(y_test, y_pred))
print("Classification Report: ", classification_report(y_test, y_pred))


Accuracy:  0.7850020772746157
Confusion Matrix:  [[2142  585]
 [ 450 1637]]
Classification Report:                precision    recall  f1-score   support

           0       0.83      0.79      0.81      2727
           1       0.74      0.78      0.76      2087

    accuracy                           0.79      4814
   macro avg       0.78      0.78      0.78      4814
weighted avg       0.79      0.79      0.79      4814



#### Modelo TF-IDF

In [19]:
#Split the data into train and test
target = df['label']

X_train_TF, X_test_TF, y_train_TF, y_test_TF = train_test_split(df_tfidf, target, test_size=0.3, random_state=42)

In [20]:
#Multinomial Naive Bayes model

model_naive_tfidf = MultinomialNB()
model_naive_tfidf.fit(X_train_TF, y_train_TF)

y_pred_TF = model_naive_tfidf.predict(X_test_TF)

#Evaluate the model
print("Accuracy: ", accuracy_score(y_test_TF, y_pred_TF))
print("Confusion Matrix: ", confusion_matrix(y_test_TF, y_pred_TF))
print("Classification Report: ", classification_report(y_test_TF, y_pred_TF))


Accuracy:  0.8564603240548401
Confusion Matrix:  [[2457  270]
 [ 421 1666]]
Classification Report:                precision    recall  f1-score   support

           0       0.85      0.90      0.88      2727
           1       0.86      0.80      0.83      2087

    accuracy                           0.86      4814
   macro avg       0.86      0.85      0.85      4814
weighted avg       0.86      0.86      0.86      4814

